<a href="https://colab.research.google.com/github/Nikhileswar-Komati/100D_ML/blob/master/Copy_of_Go_Vs_Python_FILE_IO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory

profile = ExecutionProfile(
    load_balancing_policy=WhiteListRoundRobinPolicy(['127.0.0.1']),
    retry_policy=DowngradingConsistencyRetryPolicy(),
    consistency_level=ConsistencyLevel.LOCAL_QUORUM,
    serial_consistency_level=ConsistencyLevel.LOCAL_SERIAL,
    request_timeout=15,
    row_factory=tuple_factory
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()

print(session.execute("SELECT release_version FROM system.local").one())

In [ ]:
import csv
import os
import time
import pandas as pd

# df = pd.read_csv("from_2012.csv")
# print(df.shape)
csv_reader1 = csv.reader(open("from_2012.csv"))
csv_reader2 = csv.reader(open("from_2012.csv"))
csv_reader3 = csv.reader(open("from_2012.csv"))
a = time.time()

with open('file1.csv', 'w') as csvfile1:
    csv_writer1 = csv.writer(csvfile1)
    for row in csv_reader1:
        csv_writer1.writerow(row)

b = time.time()

# with open('file2.csv', 'w') as csvfile2:
#     csv_writer2 = csv.writer(csvfile2)

#     i = 0
#     rows = []
#     for row in csv_reader2:
#         i +=1
#         rows.append(row)
#         if i==10000:
#             i = 0
#             csv_writer2.writerows(rows)
#             rows = []

# c = time.time()

# chunks = c-b
# single = b-a
# # print(c-b)
# print(chunks, single, chunks>single)

# a = time.time()

# with open('file1.csv', 'w') as csvfile1:
#     csv_writer1 = csv.writer(csvfile1)
#     csv_writer1.writerows(csv_reader3)

# b = time.time()

print(b-a)

In [ ]:
import csv
import os
import time



steps = [2, 10, 20]

ind = 0
for step in steps:
    csv_reader = csv.reader(open("from_2012.csv"))
    b = time.time()
    file_name = 'file{}.csv'.format(ind)
    with open(file_name, 'w') as csvfile2:
        csv_writer2 = csv.writer(csvfile2)

        i = 0
        rows = []
        for row in csv_reader:
            i +=1
            rows.append(row)
            if i==step:
                i = 0
                csv_writer2.writerows(rows)
                rows = []
    ind += 1
    c = time.time()

    chunks = c-b
    print(chunks, file_name)


In [ ]:
package main

import (
	"encoding/csv"
	"fmt"
	"log"
	"os"
	"time"
)

func readCsvFile(filePath string) [][]string {
	f, err := os.Open(filePath)
	if err != nil {
		log.Fatal("Unable to read input file "+filePath, err)
	}
	defer f.Close()

	csvReader := csv.NewReader(f)
	records, err := csvReader.ReadAll()
	if err != nil {
		log.Fatal("Unable to parse file as CSV for "+filePath, err)
	}

	return records
}

func writeCsvFile(records [][]string, filePath string) {
	f, err := os.Create(filePath)
	if err != nil {
		log.Fatalf("failed creating file: %s", err)
	}
	defer f.Close()

	csvwriter := csv.NewWriter(f)
	for _, record := range records {
		_ = csvwriter.Write(record)
	}
	csvwriter.Flush()
}

func writeAllCsvFile(records [][]string, filePath string) {
	f, err := os.Create(filePath)
	if err != nil {
		log.Fatalf("failed creating file: %s", err)
	}
	defer f.Close()

	csvwriter := csv.NewWriter(f)
	csvwriter.WriteAll(records)
}

func main() {
	records := readCsvFile("from_2012.csv")
	start := time.Now()
	writeCsvFile(records, "gofile.csv")
	end := time.Now()
	fmt.Println(end.Sub(start))
	start1 := time.Now()
	writeAllCsvFile(records, "goallfile.csv")
	end1 := time.Now()
	fmt.Println(end1.Sub(start1))
}


In [ ]:
package main

import (
	"fmt"
	"github.com/go-gota/gota/dataframe"
	"gonum.org/v1/gonum/mat"
	// "reflect"
	"github.com/pa-m/sklearn/preprocessing"
	"strings"
	"io/ioutil"
)

type matrix struct {
	dataframe.DataFrame
}

func (m matrix) At(i, j int) float64 {
	return m.Elem(i, j).Float()
}

func (m matrix) T() mat.Matrix {
	return mat.Transpose{Matrix: m}
}

func getNumericalColumns(df dataframe.DataFrame) dataframe.DataFrame{
	list_of_datatypes := df.Types()
	var numerical_datatype_indexes []int
	for indx, datatype := range list_of_datatypes {
		if datatype == "float" || datatype == "int" {
			numerical_datatype_indexes = append(numerical_datatype_indexes, indx)
		}
	}
	return df.Select(numerical_datatype_indexes)
}

func data_normalization_function(data dataframe.DataFrame, norm_type string, composite_columns []string) {
	fmt.Println("In Data Normalization") 
	norm_type = strings.ToLower(norm_type)
	data_without_time := data.Drop(composite_columns)
	data_num := getNumericalColumns(data_without_time)
	data_num_matrix := matrix{data_num}
	var scaled_num_data *mat.Dense
	if norm_type == "minmax" {
		scaler := preprocessing.NewMinMaxScaler([]float64{0, 1})
		scaled_num_data, _ = scaler.FitTransform(data_num_matrix, nil)
	} else if norm_type == "standard" {
		scaler := preprocessing.NewStandardScaler()
		scaled_num_data, _ = scaler.FitTransform(data_num_matrix, nil)
	} else if norm_type == "maxabs" {
		scaler := preprocessing.NewMaxAbsScaler()
		scaled_num_data, _ = scaler.FitTransform(data_num_matrix, nil)
	} else if norm_type == "robust" {
		scaler := preprocessing.NewRobustScaler(true, false, nil)
		scaled_num_data, _ = scaler.FitTransform(data_num_matrix, nil)
	}
	fmt.Println(scaled_num_data)
	scaled_num_data_frame := dataframe.LoadMatrix(scaled_num_data)
	num_column_names := data_num.Names()
	scaled_num_data_frame.SetNames(num_column_names...)
	fmt.Println(scaled_num_data_frame)
	df_without_num_columns := data.Drop(num_column_names)
	data = scaled_num_data_frame.CBind(df_without_num_columns)
	fmt.Println(data)
}

func main() {
	content, _ := ioutil.ReadFile("sample_go.csv")
	ioContent := strings.NewReader(string(content))

	df := dataframe.ReadCSV(ioContent)
	norm_type := "minmax"
	var composite_columns []string

	data_normalization_function(df, norm_type, composite_columns)
}

In [ ]:
package main

import (
	"fmt"
	"github.com/go-gota/gota/dataframe"
	"gonum.org/v1/gonum/mat"
	// "reflect"
	// "github.com/pa-m/sklearn/preprocessing"
	"strings"
	"math"
	"io/ioutil"
)

type matrix struct {
	dataframe.DataFrame
}

func (m matrix) At(i, j int) float64 {
	return m.Elem(i, j).Float()
}

func (m matrix) T() mat.Matrix {
	return mat.Transpose{Matrix: m}
}

func getNumericalColumns(df dataframe.DataFrame) dataframe.DataFrame{
	list_of_datatypes := df.Types()
	var numerical_datatype_indexes []int
	for indx, datatype := range list_of_datatypes {
		if datatype == "float" || datatype == "int" {
			numerical_datatype_indexes = append(numerical_datatype_indexes, indx)
		}
	}
	return df.Select(numerical_datatype_indexes)
}

func replaceValuesInNumericColumns(df dataframe.DataFrame, old_value float64, new_value float64) dataframe.DataFrame{
	num_df := getNumericalColumns(df)
	num_column_names := num_df.Names()
	data_num_matrix := matrix{num_df}
	dense_data_matrix := mat.DenseCopyOf(data_num_matrix)
	// fmt.Println(num_df.Names())
	// fmt.Println(dense_data_matrix)
	// fmt.Println(dense_data_matrix.IsEmpty())
	// fmt.Println(reflect.TypeOf(dense_data_matrix))
	nrows, ncols := dense_data_matrix.Dims()
	for row := 0; row < nrows; row++ {
		for col := 0; col < ncols; col++ {
			ele := dense_data_matrix.At(row, col)
			if ele == old_value || math.IsNaN(ele){
				dense_data_matrix.Set(row, col, new_value)
			}
		}
	}
	num_df = dataframe.LoadMatrix(dense_data_matrix)
	num_df.SetNames(num_column_names...)
	return num_df
}

func replaceValuesInCatColumns(df dataframe.DataFrame, old_value string, new_value string) dataframe.DataFrame {
	num_df := getNumericalColumns(df)
	cat_df := df.Drop(num_df.Names())
	cat_column_names := cat_df.Names()
	records := cat_df.Records()
	var ele string
	for row_indx, row := range records {
		for col_indx, _ := range row {
			ele = strings.ToLower(records[row_indx][col_indx])
			if ele == old_value || ele == "" {
				records[row_indx][col_indx] = new_value
			}
		}
	}
	return dataframe.LoadRecords(records, dataframe.Names(cat_column_names...))
}

func replaceValuesInDataFrame(df dataframe.DataFrame, old_value float64, new_value float64) dataframe.DataFrame {
	var df1 , df2 dataframe.DataFrame
	if math.IsNaN(old_value) {
		fmt.Println("--------------------Inside replace values in Df --------------------------")
		df1 = replaceValuesInCatColumns(df, "nan", "0")
		df2 = replaceValuesInNumericColumns(df, old_value, new_value)
	} else {
		df1 = replaceValuesInCatColumns(df, "0", "nan")
		df2 = replaceValuesInNumericColumns(df, old_value, new_value)
	}
	df = df1.CBind(df2)
	return df
}

func dropNullValuesWithThreshold(data dataframe.DataFrame, threshold_count int) dataframe.DataFrame{
	nrows, ncols := data.Dims()
	var non_null_counter int
	var keep_indexes []int 
	fmt.Println("Threshold = ", threshold_count)
	for row := 0; row < nrows; row++ {
		non_null_counter = 0
		for col :=0; col < ncols; col++{
			ele_float := data.Elem(row, col).Float()
			ele_string := data.Elem(row, col).String()
			ele_string = strings.ToLower(ele_string)
			fmt.Println(ele_float, ele_string)
			if !math.IsNaN(ele_float) || ele_string != "nan"{
				non_null_counter += 1
			} 
		}
		if non_null_counter >= threshold_count {
			//remove the row from matrix
			keep_indexes = append(keep_indexes, row)
		}
	}
	return data.Subset(keep_indexes)
}

func data_cleaning_function(data dataframe.DataFrame, cleaning_threshold int, is_zero_valid_data string) dataframe.DataFrame{
	fmt.Println("In Data Cleaning") 
	is_zero_valid_data = strings.ToLower(is_zero_valid_data)
	column_names := data.Names()
	column_length := len(column_names)
	var cleaned_data_frame dataframe.DataFrame
	threshold_count := column_length - int((column_length * cleaning_threshold)/100)
	if is_zero_valid_data == "no" {
		// fmt.Println(reflect.ValueOf(data_num_matrix))
		cleaned_data_frame = replaceValuesInDataFrame(data, 0, math.NaN())
		fmt.Println(cleaned_data_frame)
		fmt.Println("1st stage of cleaning done ")
	}
	cleaned_data_frame = dropNullValuesWithThreshold(cleaned_data_frame, threshold_count)
	fmt.Println(cleaned_data_frame)
	fmt.Println("dropping null values with threshold done")
	if is_zero_valid_data == "no" {
		cleaned_data_frame = replaceValuesInDataFrame(cleaned_data_frame, math.NaN(), 0)
		fmt.Println(cleaned_data_frame)
		fmt.Println("2nd stage of cleaning done")
	}	
	return cleaned_data_frame

}

func main() {
	content, _ := ioutil.ReadFile("sample_go.csv")
	ioContent := strings.NewReader(string(content))
	nan_values := []string {"NA", "NaN", "<nil>"}
	df := dataframe.ReadCSV(ioContent, dataframe.NaNValues(nan_values))
	fmt.Println(df)
	cleaning_threshold := 30
	is_zero_valid_data := "no"
	fmt.Println("Data Cleaning Started")
	cleaned_df := data_cleaning_function(df, cleaning_threshold, is_zero_valid_data)
	fmt.Println(cleaned_df)
	fmt.Println("Data Cleaning Done")
}

In [ ]:
package main

import (
	"fmt"
	"github.com/go-gota/gota/dataframe"
	"github.com/go-gota/gota/series"
	"gonum.org/v1/gonum/mat"
	// "reflect"
	"github.com/pa-m/sklearn/preprocessing"
	"strings"
	"io/ioutil"
	// "math"
)

type matrix struct {
	dataframe.DataFrame
}

func (m matrix) At(i, j int) float64 {
	return m.Elem(i, j).Float()
}

func (m matrix) T() mat.Matrix {
	return mat.Transpose{Matrix: m}
}

func uniqueElementsOfSlice(slice []string) []string {
    // create a map with all the values as key
    uniqMap := make(map[string]struct{})
    for _, v := range slice {
        uniqMap[v] = struct{}{}
    }
    // turn the map keys into a slice
    uniqSlice := make([]string, 0, len(uniqMap))
    for v := range uniqMap {
        uniqSlice = append(uniqSlice, v)
    }
    return uniqSlice
}

func getNumericalColumns(df dataframe.DataFrame) dataframe.DataFrame{
	list_of_datatypes := df.Types()
	var numerical_datatype_indexes []int
	for indx, datatype := range list_of_datatypes {
		if datatype == "float" || datatype == "int" {
			numerical_datatype_indexes = append(numerical_datatype_indexes, indx)
		}
	}
	return df.Select(numerical_datatype_indexes)
}

func data_imputation_function(data dataframe.DataFrame, impute_type string, composite_columns []string, is_zero_valid_data string, start_epoch int, end_epoch int) {
	fmt.Println("In Data Imputation") 
	impute_type = strings.ToLower(impute_type)
	is_zero_valid_data = strings.ToLower(is_zero_valid_data)
	data_without_time := data.Drop(composite_columns)
	data_num := getNumericalColumns(data_without_time)
	data_num_matrix := matrix{data_num}
	// fmt.Println(data_num_matrix)
	if is_zero_valid_data == "no" {
	}
	if impute_type == "mean" || impute_type == "median" || impute_type == "most_frequent" {
		imputed_model := &preprocessing.Imputer{Strategy: impute_type}
		imputed_num_data, _ := imputed_model.FitTransform(data_num_matrix, nil)
		
		// fmt.Println(imputed_num_data)
		imputed_num_data_frame := dataframe.LoadMatrix(imputed_num_data)
		// fmt.Println(imputed_num_data_frame)
		num_column_names := data_num.Names()
		imputed_num_data_frame.SetNames(num_column_names...)
		// fmt.Println(imputed_num_data_frame)
		df_without_num_columns := data.Drop(num_column_names)
		data = imputed_num_data_frame.CBind(df_without_num_columns)
		fmt.Println(data)
	} else if impute_type == "medianoflast3days" || impute_type == "medianoflast3mins" {
		// enbid_series := data.Col("ENBID")
		enbid_series := data.Col("roll_num")
		enbids := enbid_series.Records()
		// fmt.Println(uniqueElementsOfSlice(enbids))
		unique_enbids := uniqueElementsOfSlice(enbids)
		for _, enbid := range unique_enbids {
			enb_data := data.Filter(dataframe.F{
				Colname: "ENBID",
				Comparator: series.Eq,
				Comparando: enbid,
			})
			cells_series := enb_data.Col("CELLID")
			cells := cells_series.Records()
			unique_cells := uniqueElementsOfSlice(cells)
			for _, cell := range unique_cells {
				df_data_num := data_import_cell_function(enb_data, start_epoch, end_epoch, impute_type, enbid, cell)
			}
		}
	}
	
}

func main() {
	content, _ := ioutil.ReadFile("sample_go.csv")
	ioContent := strings.NewReader(string(content))

	df := dataframe.ReadCSV(ioContent)
	var impute_type, is_zero_valid_data string
	var composite_columns []string
	var start_epoch, end_epoch int
	impute_type = "medianoflast3days"

	data_imputation_function(df, impute_type, composite_columns, is_zero_valid_data, start_epoch, end_epoch)
}